# Indexing blog posts for a blog roll

This notebooks aggregates the `deathbeds` modules into a tidy `pandas.DataFrame`.  That object can be used to created a structured view of the posts.

In [2]:
    from pathlib import Path
    import deathbeds
    from pandas import DataFrame, Series, Index, to_datetime
    from IPython.display import Markdown, display
    loader = __import__('importnb').Notebook();

* glob the modules in the `deathbeds` package.

Convert a file path to and imported module

In [4]:
    def file_to_module(path):
        with loader:
            name = path.relative_to(Path(deathbeds.__file__).parent).stem.split('.', 1)[0]
            return getattr(__import__('.'.join(('deathbeds', name))), name)

* Import all of the modules.  As the modules are imported we are testing their efficacy when `__name__ != '__main__`

In [15]:
    from pytest import mark, fixture

In [16]:
    @mark.skipif(True, reason="Another blog post tells this story better")
    def _load_all_imports():
        for path in modules.index: 
            try: modules.loc[path] = file_to_module(path)
            except BaseException as e: print(path, e)

* Tidy the module information to create a blog roll

In [17]:
    is_nbconvert = 'tmp' in __import__('sys').argv[-1]

False

In [18]:
    @fixture
    def modules():
        return Series(index=Index(Path(deathbeds.__file__).parent.glob('*.ipynb')));

In [19]:
modules()

c:\users\deathbeds\deathbeds.github.io\deathbeds\2018-06-19-Github-Statistics.ipynb                                NaN
c:\users\deathbeds\deathbeds.github.io\deathbeds\2018-06-19-String-Node-Transformer.ipynb                          NaN
c:\users\deathbeds\deathbeds.github.io\deathbeds\2018-06-20-Interactive-type-checking-with-datashape.ipynb         NaN
c:\users\deathbeds\deathbeds.github.io\deathbeds\2018-06-20-OpenCV-and-a-Widget.ipynb                              NaN
c:\users\deathbeds\deathbeds.github.io\deathbeds\2018-06-22-Show-all-outputs-in-widget.ipynb                       NaN
c:\users\deathbeds\deathbeds.github.io\deathbeds\2018-06-23-Emojis-in-matplotlib.ipynb                             NaN
c:\users\deathbeds\deathbeds.github.io\deathbeds\2018-06-23-Image-segmentation-on-flourishes.ipynb                 NaN
c:\users\deathbeds\deathbeds.github.io\deathbeds\2018-06-24-Interactive-type-checking.ipynb                        NaN
c:\users\deathbeds\deathbeds.github.io\deathbeds

In [11]:
    @fixture
    def df(modules):
        df = modules.to_frame('module')
        df['docstring'] = df['module'].apply(lambda x: x.__doc__)
        df['name'] = df['module'].apply(lambda x: x.__name__)
        df['title'] = df.docstring.str.lstrip('#').str.lstrip().apply(lambda x: x.splitlines()[0])
        df['relative'] = df.index.map(lambda x: f'{x.relative_to(Path(deathbeds.__file__).parent)}')

        df['date'] = df['name'].str.split('.', 1).apply(lambda _: '-'.join(_[1].split('-', 3)[:3])).pipe(
            lambda s: s[s.apply(lambda x: x[0]).str.isnumeric()]
        ).pipe(to_datetime)
        return df

    

In [11]:
    def create_blog_roll(df):
        return Markdown(df.dropna(subset=['date']).sort_values('date', ascending=False).apply(
            lambda _: f"""* ## [{_.loc['title']}]({_.loc['relative']})

    > {_.loc['date'].strftime("%A  %B %d, %Y")}

    """, axis=1
        ).pipe("".join));
        

In [11]:
    def _show_blog_rool(df):
        assert create_blog_roll(df)

    if __name__ == '__main__' and not is_nbconvert: 
        df = create_blog_roll()
        display(df.sample(2).reset_index(drop=True))

    if __name__ == '__main__' and not is_nbconvert: 
        display(blog_roll)

## Infer the `deathbeds` requirements.

All of the modules have been imported into the current context.  That means with a few logic circuits we could recreate the requirements for all of the notebooks in `deathbeds`.

* `mappings` renames import names to their package name.

In [9]:
    mappings = dict(
        PIL='pillow',
        skimage='scikit-image', 
        yaml= 'pyyaml',
        delegator='delegator.py',
        pidgin="git+https://github.com/deathbeds/pidgin",
        importnb="git+https://github.com/deathbeds/importnb",
        depfinder="git+https://github.com/deathbeds/depfinder@ipython-code-transform",
        **{
            'cv2.cv2': 'opencv-python', 
        }
    );

* there are some [Jupyter-console incompatible with prompt-toolkit 2.0.2 #158](https://github.com/jupyter/jupyter_console/issues/158)

In [10]:
    mappings.update(prompt_toolkit='prompt-toolkit==1.0.15')

* some of the imports we flat out don't want.

In [11]:
    nah = 'sphinxcontrib', 'pkg_resources', 'dateutil', 'storemagic', 'OpenSSL', 'socks', 'gmpy2', 'blib2to3', 'deathbeds';

* special case

In [12]:
    special = "cv2.cv2",;

### Logic

* The module has a `__file__` attribute; builtins don't
* The module is top level; it doesn't have a `.`
* The module name doesn't start with an `_`
* The module is install `install` or `develop`; in `site-packages` or the working directories.

In [13]:
    requirements = ""

    for key, module in __import__('sys').modules.items():
        file = getattr(module, '__file__', "")
        if module.__name__ in special or (
            '.' not in key 
            and '.' not in module.__name__
            and not key.startswith('_')
            and (
                all(map(file.__contains__, ('python', 'site-packages')))
                or 'python' not in getattr(module, '__file__', "python")
            ) and module.__name__ not in nah
        ):
            requirements += f"""{mappings.get(module.__name__, module.__name__)}\n"""

    requirements = '\n'.join(sorted(requirements.splitlines(), key=str.lower));
    Path('../requirements.txt').write_text(requirements)
    print(requirements)

appdirs
asn1crypto
attr
black
bottleneck
certifi
cffi
chardet
click
cloudpickle
colorama
CommonMark
cryptography
cycler
cytoolz
dask
dataclasses
datashape
decorator
delegator.py
entrypoints
fastcache
flask
git+https://github.com/deathbeds/depfinder@ipython-code-transform
git+https://github.com/deathbeds/importnb
git+https://github.com/deathbeds/pidgin
google
graphviz
greenlet
idna
importlib_resources
ipykernel
ipyparallel
IPython
ipython_genutils
ipywidgets
itsdangerous
jedi
jinja2
jsonschema
jupyter_client
jupyter_core
luigi
lxml
markupsafe
matplotlib
mistune
more_itertools
mpmath
multipledispatch
mypy
nbconvert
nbformat
notebook
numpy
numpydoc
opencv-python
opencv-python
pandas
pandocfilters
pexpect
pickleshare
pillow
pluggy
poser
prompt-toolkit==1.0.15
ptyprocess
py
pygments
pyparsing
pytest
pytz
pywt
pyyaml
requests
scikit-image
scipy
shapely
simplegeneric
simplejson
six
stdlib_list
sympy
tensorflow
testpath
toolz
tornado
tqdm
traitlets
typed_ast
urllib3
wcwidth
werkzeug
xlrd
zmq
